# Tensorflow를 이용한 MNIST 실습 

In [41]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [42]:
tf.reset_default_graph()

In [43]:
mnist = input_data.read_data_sets('data/mnist/', one_hot=True)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


In [44]:
mnist.train, mnist.test

(<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet at 0x1360ce10>,
 <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet at 0x1369efd0>)

In [45]:
X = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

In [46]:
W_1 = tf.Variable(tf.random_normal([784, 128]))
b_1 = tf.Variable(tf.random_normal([128]))
hidden_1 = tf.sigmoid(tf.matmul(X, W_1) + b_1) #logistic 함수와 모양이 같다 한방에 128개를 sigmoid 보내버림

W_2 = tf.Variable(tf.random_normal([128, 64]))
b_2 = tf.Variable(tf.random_normal([64]))
hidden_2 = tf.sigmoid(tf.matmul(hidden_1, W_2) + b_2)

W_3 = tf.Variable(tf.random_normal([64, 10]))
b_3 = tf.Variable(tf.random_normal([10]))
hypo = tf.sigmoid(tf.matmul(hidden_2, W_3) + b_3)

In [53]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        logits=hypo, labels=y)) #hypo 써서 cost 만들기

In [54]:
learning_rate = 1 #그래프에 팅기는 애
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)  
#그라디엔 써서 cost를 옵티마이징시킴

predict : hypo > 0.5 크다면 1로 저장.
hypo > 0.5 -> 논리식이기 때문에, 결과값이 True or False
tf.cast("value", "변환할 type") : value를 해당 type으로 형변환 : True -> 1, False -> 0  
    
predict : hypo > 0.5 크다면 1로 저장.
hypo > 0.5 -> 논리식이기 때문에, 결과값이 True or False
tf.cast("value", "변환할 type") : value를 해당 type으로 형변환 : True -> 1, False -> 0      

In [55]:
predict = tf.cast(hypo > 0.5, tf.float32) # 1 or 0      #cast는 형변환
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, y), tf.float32))

In [56]:
batch_size = 100
epoches = 10

In [57]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) #위에 정의된 신경망 모델 실행

    # total_batch 수(업데이트 수) = 전체 데이터 개수 // batch크기
    total_batch = mnist.train.num_examples//batch_size
    print('total_batch : ', total_batch)

    for epoch in range(epoches):
        avg_cost = 0 # 에폭 한번 돌리고 cost 바뀌는 것을 그냥 에폭 단위로 평균 돌림

        for i in range(total_batch):
            #training dataset
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)


            sess.run(optimizer,   # 만들어 놓은 opti에서 스위치를 만듦.
                     feed_dict={X: batch_xs, y: batch_ys})
            c = sess.run(cost
                         , feed_dict={X: batch_xs, y: batch_ys})    
            #버튼 눌러서 배치 사이즈의 시점에서 코스트 값 담음

            avg_cost += c / total_batch # 자기 자신에 C를 더함

        print('{:2} : {:}'.format(epoch+1, avg_cost)) 
    print('Learning finished!') #출력

    print('accuracy :', sess.run(accuracy, feed_dict={X: mnist.test.images, 
                                                  y: mnist.test.labels})) 
    #placeholder인 y를 위해 이 값을 가져옴
    #testset의 애들을 갖고와서 물어봐줌

total_batch :  550
 1 : 1.9119161653518677
 2 : 1.6852922693165857
 3 : 1.6343425156853417
 4 : 1.608605131886221
 5 : 1.5920503690026029
 6 : 1.5799270131371241
 7 : 1.5704823290218008
 8 : 1.5632699732346953
 9 : 1.5568278685483055
10 : 1.5517037120732406
Learning finished!
accuracy : 0.96801


# 새거!!!!!!!!!

### 핵심은  정확도 비교해보기

1) non-linearity 바꿔보기 : sigmoid -> relu  
2) weight 초기화 바꿔보기: random -> xavier  

In [72]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [73]:
tf.reset_default_graph()

In [74]:
mnist= input_data.read_data_sets('data/mnist/', one_hot =True)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


In [75]:
mnist.train, mnist.test

(<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet at 0x172dfba8>,
 <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet at 0x172dfd30>)

In [76]:
X= tf.placeholder(tf.float32)
y= tf.placeholder(tf.float32)

neural networks 구축  
1) weight 초기화 바꿔보기  
2) non-linearity 바꿔보기  

non-linearity : sigmoid or tanh  
-> initializer : Xavier  
non-linearity : relu  
-> initializer : He = variance_scaling   
  
bias = 0으로 초기화하자  


In [77]:
W_1 = tf. Variable(tf.random_normal([784,128]))
b_1 = tf. Variable(tf.random_normal([128]))
hidden_1 = tf. sigmoid(tf.matmul(X, W_1)+b_1)

W_2 = tf.Variable(tf.random_normal([128, 10]))
b_2 = tf.Variable(tf.random_normal([10]))
hypo = tf. sigmoid(tf.matmul(hidden_1, W_2)+ b_2)

In [79]:
W_1 = tf.get_variable('w1', shape =[784,128],
                     initializer=tf.contrib.layers.xavier_initializer())
b_1 = tf.get_variable('b1', shape =[128],
                     initializer=tf.contrib.layers.xavier_initializer())
hidden_1 = tf.nn.relu(tf.matmul(X,W_1)+b_1)

W_2 = tf.get_variable('w3', shape =[128, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b_2 = tf.get_variable('b3', shape =[10],
                     initializer=tf.contrib.layers.xavier_initializer())
hypo = tf.matmul(hidden_1,W_2)+b_2

In [81]:
cost =tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        logits=hypo, labels=y))
learning_rate=.1
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


In [82]:
predict = tf.cast(hypo > 0.5, tf.float32) # 1 or 0
acc = tf.reduce_mean(tf.cast(tf.equal(predict, y), tf.float32))

In [83]:
batch_size = 100
epoches = 10

In [86]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # total_batch 수(업데이트 수) = 전체 데이터 개수 // batch크기
    total_batch = mnist.train.num_examples//batch_size
    print('total_batch : ', total_batch)

    for epoch in range(epoches):
        avg_cost = 0

        for i in range(total_batch):
            #training dataset
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)


            sess.run(optimizer, 
                     feed_dict={X: batch_xs, y: batch_ys})
            c = sess.run(cost
                         , feed_dict={X: batch_xs, y: batch_ys})

            avg_cost += c / total_batch

        print('{:2} : {:}'.format(epoch+1, avg_cost))
    print('Learning finished!')
    
    pred = tf.equal(tf.argmax(hypo, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(pred, tf.float32))

    print('accuracy :', sess.run(accuracy, feed_dict={X: mnist.test.images, 
                                                  y: mnist.test.labels,
                                                     keep_prob:1.0}))


total_batch :  550
 1 : 0.6106235406886446
 2 : 0.43006782611662664
 3 : 0.509155262383547
 4 : 0.47617919577793666
 5 : 0.5247935067794535
 6 : 0.5847815721956173
 7 : 0.5990648125247523
 8 : 0.5770066307349644
 9 : 0.6006747571175757
10 : 0.6123743858120667
Learning finished!


NameError: name 'keep_prob' is not defined